# DPSI case study

### Notebook helper function for pretty output

In [ ]:
from ipywidgets import interact
from tabulate import tabulate

from glotaran.utils.ipython import MarkdownStr

### Plotting functions ([`pyglotaran_extras`](https://github.com/s-weigand/pyglotaran-extras/commit/20da3593105fb839f86e668dc12dc9ca87c3b9ce) + `matplotlib`)

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.style import PlotStyle

plot_style = PlotStyle()
plt.rc("axes", prop_cycle=plot_style.cycler)
plt.rcParams["figure.figsize"] = (21, 14)

### Analysis functions

In [ ]:
from glotaran.analysis.optimize import optimize
from glotaran.io import load_dataset
from glotaran.io import load_model
from glotaran.io import load_parameters
from glotaran.project.scheme import Scheme

### Read data

In [ ]:
dataset1 = load_dataset("data/RT400_590excDPSIjul30tr24KtargetPB10a.ascii")
dataset2 = load_dataset("data/RT400_590excDPSIjul30tr24KtargetPB10b.ascii")
dataset3 = load_dataset("data/RT400_590excDPSIjul30tr24KtargetPB10c.ascii")
dataset4 = load_dataset("data/RT400_590excDPSIjul30tr24KtargetPB10d.ascii")
dataset5 = load_dataset("data/RT400_590excDPSIjul30tr24KtargetPB10e.ascii")
dataset6 = load_dataset("data/RT400_590excDPSIjul30tr24KtargetPB10f.ascii")

four_datasets = {
    "dataset1": dataset1,
    "dataset2": dataset2,
    "dataset3": dataset3,
    "dataset4": dataset4,
}
six_datasets = {**four_datasets, "dataset5": dataset5, "dataset6": dataset6}


def plot_datasets(dataset_name):
    """Plot helper function to use with widgets"""
    dataset = six_datasets[dataset_name]
    if len(dataset.data.time) > 1:
        dataset.data.plot(x="time", center=False)
    else:
        dataset.data.plot()


interact(plot_datasets, dataset_name=six_datasets.keys())

## Sequential Model

### Load model and parameters

In [ ]:
sequential_model = load_model("models/model_DPSI_sequential_irf.yml")
sequential_parameters = load_parameters("models/parameters_DPSI_sequential_irf.yml")
sequential_model.validate(parameters=sequential_parameters)

In [ ]:
sequential_model

### Create scheme and optimize it

In [ ]:
sequential_scheme = Scheme(
    sequential_model,
    sequential_parameters,
    data=four_datasets,
    group_tolerance=1.9,
    non_negative_least_squares=True,
    maximum_number_function_evaluations=11,
)
sequential_result = optimize(sequential_scheme)

In [ ]:
root_mean_square_errors = []
for dataset_name in four_datasets.keys():
    root_mean_square_errors.append(
        (dataset_name, sequential_result.data[dataset_name].root_mean_square_error)
    )
MarkdownStr(tabulate(root_mean_square_errors, headers=("dataset", "RMS-Error"), tablefmt="github"))

### Result plots

In [ ]:
for dataset_name in four_datasets.keys():
    fig = plot_overview(sequential_result.data[dataset_name], linthresh=150)
    fig.suptitle(f"Result for {dataset_name!r}", fontsize=16)

In [ ]:
sequential_result.optimized_parameters

## Guidance Model

### Load model and parameters

In [ ]:
guidance_model = load_model("models/model_DPSI_guidance_no_area_irf.yml")
guidance_parameters = load_parameters("models/parameters_DPSI_guidance_no_area_irf.yml")
guidance_model.validate(parameters=guidance_parameters)

In [ ]:
guidance_model

### Create scheme and optimize it

In [ ]:
guidance_scheme = Scheme(
    guidance_model,
    guidance_parameters,
    data=six_datasets,
    group_tolerance=1.9,
    non_negative_least_squares=True,
    maximum_number_function_evaluations=11,
)
guidance_result = optimize(guidance_scheme)

In [ ]:
root_mean_square_errors = []
for dataset_name in six_datasets.keys():
    root_mean_square_errors.append(
        (dataset_name, guidance_result.data[dataset_name].root_mean_square_error)
    )
MarkdownStr(tabulate(root_mean_square_errors, headers=("dataset", "RMS-Error"), tablefmt="github"))

### Result plots

In [ ]:
for dataset_name in six_datasets.keys():
    fig = plot_overview(guidance_result.data[dataset_name], linthresh=150)
    fig.suptitle(f"Result for {dataset_name!r}", fontsize=16)

In [ ]:
sequential_result.optimized_parameters

## Format notebook

In [ ]:
# !nbqa black DPSI_case_study.ipynb --nbqa-mutate
# !nbqa isort DPSI_case_study.ipynb --nbqa-mutate